# TensorBoard 可视化工具

在数据处理过程中，用户通常想要可以直观查看$\color{#ea4339}{数据集}$的分布情况。

在模型设计过程中，用户需要分析和检查$\color{#ea4339}{数据流图}$是否正确实现。

在模型训练过程中，用户也需要关注$\color{#ea4339}{模型参数}$和$\color{#ea4339}{超参数}$变化趋势。

在模型测试过程中，用户也要查看$\color{#ea4339}{准确率}$和$\color{#ea4339}{召回率}$等评估指标。

因此，$TensorFlow$项目组开发了机器学习可视化工具$\color{#ea4339}{TensorBoard}$,它通过展示直观的图形，能够有效地辅助机器学习程序的开发者和使用者理解算法模型及其工作流程，提升模型开发工作效率。

### TensorBoard 可视化训练

<img width=80% height=80% src="imgs/05/01.png" alt="imgs/05/01.png" title="图1" />

### TensorBoard 可视化统计数据

<img width=80% height=80% src="imgs/05/02.png" alt="imgs/05/02.png" title="图2" />

### TensorBoard 可视化数据分布

<img width=80% height=80% src="imgs/05/03.png" alt="imgs/05/03.png" title="图3" />

### TensorBoard 可视化数据集（MNIST）

<img width=80% height=80% src="imgs/05/04.png" alt="imgs/05/04.png" title="图4" />

### TensorBoard 可视化数据流图

<img width=80% height=80% src="imgs/05/05.png" alt="imgs/05/05.png" title="图5" />

### TensorBoard 使用流程

可视化的数据是数据流图和张量，它们需要在会话中加载或执行操作后才能获取。然后，用户需要使用$\color{#ea4339}{FileWriter}$实例将这些数据写入事件文件。最后，启动$TensorBoard$程序，加载事件文件中的序列化数据，从而可以在各个面板中展示对应的可视化对象。

<img width=80% height=80% src="imgs/05/06.png" alt="imgs/05/06.png" title="图6" />

### tf.summary 模块介绍

前述流程中使用的FileWriter实例和汇总操作(Summary Ops)均属于tf.summary模块。其主要功能是获取和输出模型相关的序列化数据，它贯通TensorBoard的整个使用流程。

tf.summary模块的核心部分由一组汇总操作以及FileWriter、Summary和Event 3个类组成。

<img width=60% height=60% src="imgs/05/07.png" alt="imgs/05/07.png" title="图7" />

# 可视化数据流图 工作流

<img width=80% height=80% src="imgs/05/08.png" alt="imgs/05/08.png" title="图8" />

#### Which one is better ?

<img width=80% height=80% src="imgs/05/09.png" alt="imgs/05/09.png" title="图9" />

### 名字作用域与抽象节点


In [3]:
#Numpy 数据处理
import pandas as pd
import numpy as np

def normalize_feature(df):
    return df.apply(lambda column: (column - column.mean()) / column.std())

df = normalize_feature(pd.read_csv('datas/04/train_3d.csv', names=['square', 'bedrooms', 'price']))

ones = pd.DataFrame({'ones': np.ones(len(df))}) #ones是n行1列的数据框，表示x0恒为1
df = pd.concat([ones,df], axis=1) #根据列合并数据

X_data = np.array(df[df.columns[0:3]]) #取的0-2列，即前三列；左闭右开的区间
y_data = np.array(df[df.columns[-1]]).reshape(len(df), 1) #-1就是最后一列

print(X_data.shape, type(X_data))
print(y_data.shape, type(y_data))

(10000, 3) <class 'numpy.ndarray'>
(10000, 1) <class 'numpy.ndarray'>


In [4]:
import tensorflow as tf

alpha = 0.01 # 学习率
epoch = 400 # 训练全量数据集的轮数

# 创建线性回归模型

with tf.name_scope('input'):
    # 输入 X，形状[10000,3]
    X = tf.placeholder(tf.float32, X_data.shape)
    # 输入 X，形状[10000,1]
    y = tf.placeholder(tf.float32, y_data.shape)

with tf.name_scope('hypothesis'):
    # weight [3, 1]
    W = tf.get_variable('weights',(X_data.shape[1], 1), initializer=tf.constant_initializer())
    # 假设函数 h(x) = w_0 * x_0 + w_1 * x_1 + w_2 * x_2, 其中x_0恒为1
    # 推理值 y_pred [1000,1]
    y_pred = tf.matmul(X, W)

with tf.name_scope('loss'):
    # 损失函数采用最小二乘法，y_pred - y 是形如[1000,1]的向量
    # tf.matmul(a, b, transpose_a=True) 表示：矩阵a的转置乘矩阵b，即 [1,1000] x [1000,1]
    # 损失函数操作 loss
    loss_op = 1 / (2 * len(X_data)) * tf.matmul((y_pred - y), (y_pred - y), transpose_a=True)

with tf.name_scope('train'):
    # 随机梯度下降优化器 opt
    opt = tf.train.GradientDescentOptimizer(learning_rate=alpha)
    
    # 单步训练操作 train_op
    train_op = opt.minimize(loss_op)

/Users/po/Documents/repo/venv/py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/po/Documents/repo/venv/py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/po/Documents/repo/venv/py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/po/Documents/repo/venv/py3/lib/python3.6/s

In [5]:
# create session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # save sess graph
    writer = tf.summary.FileWriter('./summary/linear-regression-5-1', sess.graph)
    
    for e in range(1, epoch + 1):
        sess.run(train_op, feed_dict={X: X_data, y: y_data})
        if e % 10 == 0:
            loss, w = sess.run([loss_op, W], feed_dict={X: X_data, y: y_data})
            log_str = "Epoch_%d \t Loss=%.4g \t Model: y = %.4gx1 + %.4gx2 + %.4g"
            print(log_str % (e, loss, w[1], w[2], w[0]))
            
# close file writer
writer.close()

Epoch_10 	 Loss=0.4596 	 Model: y = 0.05679x1 + 0.02882x2 + 2.765e-10
Epoch_20 	 Loss=0.4291 	 Model: y = 0.1069x1 + 0.0524x2 + 3.658e-10
Epoch_30 	 Loss=0.4059 	 Model: y = 0.1513x1 + 0.07152x2 + 3.859e-10
Epoch_40 	 Loss=0.3881 	 Model: y = 0.1906x1 + 0.08687x2 + 3.606e-10
Epoch_50 	 Loss=0.3746 	 Model: y = 0.2255x1 + 0.09902x2 + 1.516e-10
Epoch_60 	 Loss=0.3641 	 Model: y = 0.2566x1 + 0.1085x2 + -4.278e-11
Epoch_70 	 Loss=0.3559 	 Model: y = 0.2843x1 + 0.1157x2 + -2.494e-10
Epoch_80 	 Loss=0.3495 	 Model: y = 0.309x1 + 0.1209x2 + -3.376e-10
Epoch_90 	 Loss=0.3445 	 Model: y = 0.3312x1 + 0.1246x2 + -3.82e-10
Epoch_100 	 Loss=0.3405 	 Model: y = 0.3511x1 + 0.1269x2 + -5.777e-10
Epoch_110 	 Loss=0.3373 	 Model: y = 0.3691x1 + 0.1282x2 + -7.139e-10
Epoch_120 	 Loss=0.3347 	 Model: y = 0.3853x1 + 0.1285x2 + -9.823e-10
Epoch_130 	 Loss=0.3325 	 Model: y = 0.3999x1 + 0.1281x2 + -1.124e-09
Epoch_140 	 Loss=0.3308 	 Model: y = 0.4132x1 + 0.127x2 + -1.142e-09
Epoch_150 	 Loss=0.3293 	 Model:

### 启动 TensorBoard

<img width=80% height=80% src="imgs/05/10.png" alt="imgs/05/10.png" title="图10" />